## 📊 Notebook 3: Dashboard SQL Queries for Power BI

#### Goal: Structured SQL queries to extract business-ready insights for Power BI Dashboard from cleaned PostgreSQL database.

In [1]:
# ✅ 1. Import Libraries and Database Connection
import pandas as pd
from sqlalchemy import create_engine

DB_NAME = "telco_churn_db"
DB_USER = "hridyanshkatal"
DB_HOST = "localhost"
DB_PORT = "5432"

engine = create_engine(f'postgresql://{DB_USER}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# ✅ Helper Function to Export Query Results to CSV
def export_query(query, file_name):
    df = pd.read_sql(query, engine)
    df.to_csv(f'../data/dashboard_exports/{file_name}.csv', index=False)
    print(f"Exported: {file_name}.csv")
    return df

### 📊 Basic Churn Overview

#### Goal: Capture overall churn numbers, churn %, total customers — useful for dashboard cards.

#### Query 1: Overall Churn Count and Percentage

In [5]:
query1 = """
SELECT 
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM cleaned_telco_churn), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY Churn
ORDER BY Churn;
"""

df_churn_overview = pd.read_sql(query1, engine)
export_query(query1, "overall_churn_distribution")
df_churn_overview


Exported: overall_churn_distribution.csv


,churn,customer_count,churn_percentage
0,No,5163,73.42
1,Yes,1869,26.58


✅ Insight: Gives total number of churned vs retained customers and overall churn rate.

#### Query 2: Total Customer Count (for reference)

In [7]:
query2 = """
SELECT COUNT(*) AS total_customers FROM cleaned_telco_churn;
"""
df_total_customers = pd.read_sql(query2, engine)
export_query(query2, "total_customers")
df_total_customers


Exported: total_customers.csv


,total_customers
0,7032


In [8]:
# ✅ Query 3: Churn by Senior Citizen Status
query3 = """
SELECT 
    SeniorCitizen,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY SeniorCitizen), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY SeniorCitizen, Churn
ORDER BY SeniorCitizen, Churn;
"""

df_senior_churn = export_query(query3, 'senior_churn_distribution.csv')
df_senior_churn


Exported: senior_churn_distribution.csv.csv


,seniorcitizen,churn,customer_count,churn_percentage
0,0,No,4497,76.35
1,0,Yes,1393,23.65
2,1,No,666,58.32
3,1,Yes,476,41.68


Insight:

Senior Citizens churn significantly more (~40-42%), highlighting an age-related risk group.

Useful for pie/bar charts, or highlighting risky customer profiles in Power BI.

### 📊 Query 4: Churn by Partner and Dependents

In [9]:
# ✅ Query 4: Partner & Dependents vs Churn
query4 = """
SELECT 
    Partner,
    Dependents,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Partner, Dependents), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY Partner, Dependents, Churn
ORDER BY Partner, Dependents, Churn;
"""

df_partner_dependents_churn = export_query(query4, 'partner_dependents_churn')
df_partner_dependents_churn


Exported: partner_dependents_churn.csv


,partner,dependents,churn,customer_count,churn_percentage
0,No,No,No,2157,65.76
1,No,No,Yes,1123,34.24
2,No,Yes,No,282,78.55
3,No,Yes,Yes,77,21.45
4,Yes,No,No,1233,74.59
5,Yes,No,Yes,420,25.41
6,Yes,Yes,No,1491,85.69
7,Yes,Yes,Yes,249,14.31


Insight:

No Partner + No Dependents churn highest.

This combination often shows low engagement, useful for targeting personalized offers.

### 📊 Query 5: Churn by Gender

In [10]:
# ✅ Query 5: Churn by Gender
query5 = """
SELECT 
    gender,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY gender), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY gender, Churn
ORDER BY gender, Churn;
"""

df_gender_churn = export_query(query5, 'gender_churn_distribution')
df_gender_churn


Exported: gender_churn_distribution.csv


,gender,churn,customer_count,churn_percentage
0,Female,No,2544,73.04
1,Female,Yes,939,26.96
2,Male,No,2619,73.80
3,Male,Yes,930,26.20


 Insight:

Churn rate nearly equal across genders → no gender bias in churn.

Useful for demographic pie/bar charts.

#### 📊 Query 6: Contract Type vs Churn

In [11]:
# ✅ Query 6: Churn by Contract Type
query6 = """
SELECT 
    Contract,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Contract), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY Contract, Churn
ORDER BY Contract, Churn;
"""

df_contract_churn = export_query(query6, 'contract_churn_distribution.csv')
df_contract_churn


Exported: contract_churn_distribution.csv.csv


,contract,churn,customer_count,churn_percentage
0,Month-to-month,No,2220,57.29
1,Month-to-month,Yes,1655,42.71
2,One year,No,1306,88.72
3,One year,Yes,166,11.28
4,Two year,No,1637,97.15
5,Two year,Yes,48,2.85


 Insight:

Month-to-month customers churn the most (40%+), long-term contracts (1-year, 2-year) have significantly lower churn.

Clear contract type segmentation, great for Power BI pie/bar charts.

#### 📊 Query 7: Payment Method vs Churn

In [12]:
# ✅ Query 7: Payment Method vs Churn
query7 = """
SELECT 
    PaymentMethod,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY PaymentMethod), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY PaymentMethod, Churn
ORDER BY PaymentMethod, Churn;
"""

df_payment_churn = export_query(query7, 'payment_method_churn_distribution')
df_payment_churn


Exported: payment_method_churn_distribution.csv


,paymentmethod,churn,customer_count,churn_percentage
0,Bank transfer (automatic),No,1284,83.27
1,Bank transfer (automatic),Yes,258,16.73
2,Credit card (automatic),No,1289,84.75
3,Credit card (automatic),Yes,232,15.25
4,Electronic check,No,1294,54.71
5,Electronic check,Yes,1071,45.29
6,Mailed check,No,1296,80.80
7,Mailed check,Yes,308,19.20


Insight:

Electronic check customers churn the most, autopay methods churn less.

Directly useful in dashboard to highlight risk groups by payment habits.

#### 📊 Query 8: Paperless Billing vs Churn

In [13]:
# ✅ Query 8: Paperless Billing vs Churn
query8 = """
SELECT 
    PaperlessBilling,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY PaperlessBilling), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY PaperlessBilling, Churn
ORDER BY PaperlessBilling, Churn;
"""

df_paperless_churn = export_query(query8, 'paperless_billing_churn_distribution')
df_paperless_churn


Exported: paperless_billing_churn_distribution.csv


,paperlessbilling,churn,customer_count,churn_percentage
0,No,No,2395,83.62
1,No,Yes,469,16.38
2,Yes,No,2768,66.41
3,Yes,Yes,1400,33.59


Insight:

Paperless Billing → Higher churn.

Combine with contract/payment insights for multivariate churn analysis.

#### 📊 Query 9: Internet Service Type vs Churn

In [14]:
# ✅ Query 9: Churn by Internet Service Type
query9 = """
SELECT 
    InternetService,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY InternetService), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY InternetService, Churn
ORDER BY InternetService, Churn;
"""

df_internet_churn = export_query(query9, 'internet_service_churn_distribution')
df_internet_churn


Exported: internet_service_churn_distribution.csv


,internetservice,churn,customer_count,churn_percentage
0,DSL,No,1957,81.00
1,DSL,Yes,459,19.00
2,Fiber optic,No,1799,58.11
3,Fiber optic,Yes,1297,41.89
4,No,No,1407,92.57
5,No,Yes,113,7.43


Insight:

Fiber optic customers churn the most, followed by DSL; No internet users churn the least.

Highly useful to target churn-prone internet users.

#### 📊 Query 10: Online Security Subscription vs Churn

In [15]:
# ✅ Query 10: Churn by Online Security Subscription
query10 = """
SELECT 
    OnlineSecurity,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY OnlineSecurity), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY OnlineSecurity, Churn
ORDER BY OnlineSecurity, Churn;
"""

df_security_churn = export_query(query10, 'online_security_churn_distribution')
df_security_churn


Exported: online_security_churn_distribution.csv


,onlinesecurity,churn,customer_count,churn_percentage
0,No,No,2036,58.22
1,No,Yes,1461,41.78
2,No internet service,No,1407,92.57
3,No internet service,Yes,113,7.43
4,Yes,No,1720,85.36
5,Yes,Yes,295,14.64


Insight:

No Online Security = much higher churn → highlights upselling opportunity.

#### 📊 Query 11: Tech Support vs Churn

In [16]:
# ✅ Query 11: Churn by Tech Support Subscription
query11 = """
SELECT 
    TechSupport,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY TechSupport), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY TechSupport, Churn
ORDER BY TechSupport, Churn;
"""

df_techsupport_churn = export_query(query11, 'techsupport_churn_distribution')
df_techsupport_churn


Exported: techsupport_churn_distribution.csv


,techsupport,churn,customer_count,churn_percentage
0,No,No,2026,58.35
1,No,Yes,1446,41.65
2,No internet service,No,1407,92.57
3,No internet service,Yes,113,7.43
4,Yes,No,1730,84.80
5,Yes,Yes,310,15.20


Insight:

No Tech Support = much higher churn.

#### Query 12: Churn by StreamingTV Subscription

In [17]:
query12 = """
SELECT 
    StreamingTV,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY StreamingTV), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY StreamingTV, Churn
ORDER BY StreamingTV, Churn;
"""

df_streamingtv_churn = export_query(query12, 'streamingtv_churn_distribution')
df_streamingtv_churn


Exported: streamingtv_churn_distribution.csv


,streamingtv,churn,customer_count,churn_percentage
0,No,No,1867,66.46
1,No,Yes,942,33.54
2,No internet service,No,1407,92.57
3,No internet service,Yes,113,7.43
4,Yes,No,1889,69.89
5,Yes,Yes,814,30.11


Insight:
Customers without StreamingTV churn ~2x higher than those who use it. Shows entertainment services reduce churn.

#### Query 13: Churn by StreamingMovies Subscription

In [18]:
query13 = """
SELECT 
    StreamingMovies,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY StreamingMovies), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY StreamingMovies, Churn
ORDER BY StreamingMovies, Churn;
"""

df_streamingmovies_churn = export_query(query13, 'streamingmovies_churn_distribution')
df_streamingmovies_churn


Exported: streamingmovies_churn_distribution.csv


,streamingmovies,churn,customer_count,churn_percentage
0,No,No,1843,66.27
1,No,Yes,938,33.73
2,No internet service,No,1407,92.57
3,No internet service,Yes,113,7.43
4,Yes,No,1913,70.05
5,Yes,Yes,818,29.95


Insight :
Similar trend with StreamingMovies → customers using it churn less.

#### Query 14: Churn by Device Protection Subscription

In [19]:
query14 = """
SELECT 
    DeviceProtection,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY DeviceProtection), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY DeviceProtection, Churn
ORDER BY DeviceProtection, Churn;
"""

df_deviceprotection_churn = export_query(query14, 'deviceprotection_churn_distribution')
df_deviceprotection_churn


Exported: deviceprotection_churn_distribution.csv


,deviceprotection,churn,customer_count,churn_percentage
0,No,No,1883,60.86
1,No,Yes,1211,39.14
2,No internet service,No,1407,92.57
3,No internet service,Yes,113,7.43
4,Yes,No,1873,77.46
5,Yes,Yes,545,22.54


Insight:
Device Protection reduces churn noticeably — a good candidate for cross-selling strategies.

#### Query 15: Churn by OnlineBackup Subscription

In [20]:
query15 = """
SELECT 
    OnlineBackup,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY OnlineBackup), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY OnlineBackup, Churn
ORDER BY OnlineBackup, Churn;
"""

df_onlinebackup_churn = export_query(query15, 'onlinebackup_churn_distribution')
df_onlinebackup_churn


Exported: onlinebackup_churn_distribution.csv


,onlinebackup,churn,customer_count,churn_percentage
0,No,No,1854,60.06
1,No,Yes,1233,39.94
2,No internet service,No,1407,92.57
3,No internet service,Yes,113,7.43
4,Yes,No,1902,78.43
5,Yes,Yes,523,21.57


Insight:
No OnlineBackup → higher churn rate.

#### Query 16: Churn by Total Services Count

In [21]:
query16 = """
SELECT 
    TotalServices,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY TotalServices), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY TotalServices, Churn
ORDER BY TotalServices, Churn;
"""

df_totalservices_churn = export_query(query16, 'totalservices_churn_distribution')
df_totalservices_churn


Exported: totalservices_churn_distribution.csv


,totalservices,churn,customer_count,churn_percentage
0,0,No,45,56.25
1,0,Yes,35,43.75
2,1,No,1338,78.85
3,1,Yes,359,21.15
4,2,No,796,67.12
5,2,Yes,390,32.88
6,3,No,613,63.52
7,3,Yes,352,36.48
8,4,No,632,68.62
9,4,Yes,289,31.38


Insight:

Fewer services (0-2) → high churn (35%+),

More services (5+) → churn drops significantly, suggesting multi-service bundling reduces churn.

#### Query 17: Churn by Tenure Group

In [32]:
query17 = """
SELECT 
    CASE 
        WHEN tenure BETWEEN 0 AND 6 THEN '0-6 Months'
        WHEN tenure BETWEEN 7 AND 12 THEN '7-12 Months'
        WHEN tenure BETWEEN 13 AND 24 THEN '1-2 Years'
        WHEN tenure BETWEEN 25 AND 48 THEN '2-4 Years'
        ELSE '4+ Years'
    END AS tenure_group,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY CASE 
        WHEN tenure BETWEEN 0 AND 6 THEN '0-6 Months'
        WHEN tenure BETWEEN 7 AND 12 THEN '7-12 Months'
        WHEN tenure BETWEEN 13 AND 24 THEN '1-2 Years'
        WHEN tenure BETWEEN 25 AND 48 THEN '2-4 Years'
        ELSE '4+ Years'
    END), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY tenure_group,Churn
ORDER BY tenure_group, Churn;
    """

df_tenure_churn = export_query(query17, 'tenure_group_churn_distribution')
df_tenure_churn


Exported: tenure_group_churn_distribution.csv


,tenure_group,churn,customer_count,churn_percentage
0,0-6 Months,No,686,46.67
1,0-6 Months,Yes,784,53.33
2,1-2 Years,No,730,71.29
3,1-2 Years,Yes,294,28.71
4,2-4 Years,No,1269,79.61
5,2-4 Years,Yes,325,20.39
6,4+ Years,No,2026,90.49
7,4+ Years,Yes,213,9.51
8,7-12 Months,No,452,64.11
9,7-12 Months,Yes,253,35.89


Insight:

0-6 months churn rates are the highest (~45-50%).

Customers with tenure >2 years churn the least (~15-20%).

Highlights need for First 90 Days Retention Strategy.

#### Query 17: Monthly Charges Grouping vs Churn

In [36]:
# ✅ Query 17: Monthly Charges Grouping vs Churn
query17 = """
SELECT 
    CASE 
        WHEN MonthlyCharges < 35 THEN 'Low (<35)'
        WHEN MonthlyCharges BETWEEN 35 AND 75 THEN 'Medium (35-75)'
        WHEN MonthlyCharges > 75 THEN 'High (>75)'
    END AS monthly_group,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY 
        CASE 
            WHEN MonthlyCharges < 35 THEN 'Low (<35)'
            WHEN MonthlyCharges BETWEEN 35 AND 75 THEN 'Medium (35-75)'
            WHEN MonthlyCharges > 75 THEN 'High (>75)'
        END
    ), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY monthly_group, Churn
ORDER BY monthly_group, Churn;
"""
df_monthly_charges_churn = export_query(query17, 'monthly_charges_churn')
df_monthly_charges_churn


Exported: monthly_charges_churn.csv


,monthly_group,churn,customer_count,churn_percentage
0,High (>75),No,2038,65.34
1,High (>75),Yes,1081,34.66
2,Low (<35),No,1537,89.10
3,Low (<35),Yes,188,10.90
4,Medium (35-75),No,1588,72.58
5,Medium (35-75),Yes,600,27.42


High billing customers churn more aggressively.

Medium range customers are more stable.

Useful for pricing-tier churn analysis in dashboard charts.

#### Query 18: Total Charges Grouping vs Churn

In [37]:
# ✅ Query 18: Total Charges Grouping vs Churn
query18 = """
SELECT 
    CASE 
        WHEN TotalCharges < 1000 THEN 'Low Spend (<$1k)'
        WHEN TotalCharges BETWEEN 1000 AND 3000 THEN 'Medium ($1k-$3k)'
        WHEN TotalCharges BETWEEN 3000 AND 5000 THEN 'High ($3k-$5k)'
        ELSE 'Very High (>$5k)'
    END AS total_group,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY 
        CASE 
            WHEN TotalCharges < 1000 THEN 'Low Spend (<$1k)'
            WHEN TotalCharges BETWEEN 1000 AND 3000 THEN 'Medium ($1k-$3k)'
            WHEN TotalCharges BETWEEN 3000 AND 5000 THEN 'High ($3k-$5k)'
            ELSE 'Very High (>$5k)'
        END
    ), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY total_group, Churn
ORDER BY total_group, Churn;
"""
df_total_charges_churn = export_query(query18, 'total_charges_churn')
df_total_charges_churn


Exported: total_charges_churn.csv


,total_group,churn,customer_count,churn_percentage
0,High ($3k-$5k),No,876,81.95
1,High ($3k-$5k),Yes,193,18.05
2,Low Spend (<$1k),No,1823,63.01
3,Low Spend (<$1k),Yes,1070,36.99
4,Medium ($1k-$3k),No,1486,76.80
5,Medium ($1k-$3k),Yes,449,23.20
6,Very High (>$5k),No,978,86.17
7,Very High (>$5k),Yes,157,13.83


Insight:

Customers with low total charges churn the most, indicating early-stage churn is high.

Very High spenders churn very little, indicating loyal high-value customers.

#### Query 19: Overall Distribution of Total Services

In [38]:
# ✅ Query 20: Total Services Distribution (Count %)
query20 = """
SELECT 
    TotalServices,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM cleaned_telco_churn), 2) AS customer_percentage
FROM cleaned_telco_churn
GROUP BY TotalServices
ORDER BY TotalServices;
"""
df_total_services_distribution = export_query(query20, 'total_services_distribution')
df_total_services_distribution


Exported: total_services_distribution.csv


,totalservices,customer_count,customer_percentage
0,0,80,1.14
1,1,1697,24.13
2,2,1186,16.87
3,3,965,13.72
4,4,921,13.10
5,5,906,12.88
6,6,674,9.58
7,7,395,5.62
8,8,208,2.96


#### Query 21: High-Risk Segment Identification
Criteria:

Month-to-Month Contract

Paperless Billing

Electronic Check Payment

Tenure ≤ 12 Months

In [41]:
query21 = """
SELECT 
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) 
                               FROM cleaned_telco_churn 
                               WHERE Contract = 'Month-to-month'
                               AND PaperlessBilling = 'Yes'
                               AND PaymentMethod = 'Electronic check'
                               AND tenure <= 12), 2) AS segment_percentage
FROM cleaned_telco_churn
WHERE Contract = 'Month-to-month'
  AND PaperlessBilling = 'Yes'
  AND PaymentMethod = 'Electronic check'
  AND tenure <= 12
GROUP BY Churn
ORDER BY Churn;

"""

df_high_risk_segment = export_query(query21, "high_risk_segment_distribution")
df_high_risk_segment


Exported: high_risk_segment_distribution.csv


,churn,customer_count,segment_percentage
0,No,226,32.24
1,Yes,475,67.76


Insight:
This segment churns at 50%+ rate, signaling a priority retention group.

#### Query 21: Contract Type + Internet Service vs Churn

In [42]:
query21 = """
SELECT 
    Contract,
    InternetService,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Contract, InternetService), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY Contract, InternetService, Churn
ORDER BY Contract, InternetService, Churn;
"""

df_contract_internet_churn = export_query(query21, 'contract_internet_churn')
df_contract_internet_churn


Exported: contract_internet_churn.csv


,contract,internetservice,churn,customer_count,churn_percentage
0,Month-to-month,DSL,No,829,67.78
1,Month-to-month,DSL,Yes,394,32.22
2,Month-to-month,Fiber optic,No,966,45.39
3,Month-to-month,Fiber optic,Yes,1162,54.61
4,Month-to-month,No,No,425,81.11
5,Month-to-month,No,Yes,99,18.89
6,One year,DSL,No,517,90.70
7,One year,DSL,Yes,53,9.30
8,One year,Fiber optic,No,435,80.71
9,One year,Fiber optic,Yes,104,19.29


Insight:

Month-to-month + Fiber optic combination churns the most (often above 45-50%), while long-term contracts with DSL/No Internet show much lower churn.

#### Query 22: Monthly Charges Group + Contract Type vs Churn

In [43]:
query22 = """
SELECT 
    CASE 
        WHEN MonthlyCharges < 35 THEN 'Low (<35)'
        WHEN MonthlyCharges BETWEEN 35 AND 75 THEN 'Medium (35-75)'
        WHEN MonthlyCharges > 75 THEN 'High (>75)'
    END AS monthly_group,
    Contract,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY CASE 
        WHEN MonthlyCharges < 35 THEN 'Low (<35)'
        WHEN MonthlyCharges BETWEEN 35 AND 75 THEN 'Medium (35-75)'
        WHEN MonthlyCharges > 75 THEN 'High (>75)'
    END , Contract), 2) AS churn_percentage
FROM cleaned_telco_churn
GROUP BY monthly_group, Contract, Churn
ORDER BY monthly_group, Contract, Churn;
"""

df_monthly_contract_churn = export_query(query22, 'monthly_contract_churn')
df_monthly_contract_churn


Exported: monthly_contract_churn.csv


,monthly_group,contract,churn,customer_count,churn_percentage
0,High (>75),Month-to-month,No,837,47.34
1,High (>75),Month-to-month,Yes,931,52.66
2,High (>75),One year,No,525,82.03
3,High (>75),One year,Yes,115,17.97
4,High (>75),Two year,No,676,95.08
5,High (>75),Two year,Yes,35,4.92
6,Low (<35),Month-to-month,No,533,75.71
7,Low (<35),Month-to-month,Yes,171,24.29
8,Low (<35),One year,No,372,96.88
9,Low (<35),One year,Yes,12,3.13


Insight:

📌 High Risk Zone: Customers with High monthly charges + Month-to-Month contracts churn at alarming rates (~50%).

📌 Low Risk Zone: Low charges + Long-term contracts show significantly low churn.
This helps in financial segmentation targeting.

### Revenue Loss Queries

#### Query 23: Total Monthly Revenue Loss due to Churn

In [63]:
query23 = """
SELECT 
    SUM(MonthlyCharges) AS monthly_revenue_loss,
    ROUND((SUM(MonthlyCharges) * 100.0 / (SELECT SUM(MonthlyCharges) FROM cleaned_telco_churn))::numeric, 2) AS revenue_loss_percentage
FROM cleaned_telco_churn
WHERE Churn = 'Yes';
"""

df_revenue_loss = export_query(query23, 'total_monthly_revenue_loss')
df_revenue_loss

Exported: total_monthly_revenue_loss.csv


,monthly_revenue_loss,revenue_loss_percentage
0,139130.85,30.53


#### Query 24: Monthly Loss by Contract Type

In [64]:
query24 = """
SELECT 
    Contract,
    SUM(MonthlyCharges) AS monthly_revenue_loss,
    ROUND((SUM(MonthlyCharges) * 100.0 / (SELECT SUM(MonthlyCharges) FROM cleaned_telco_churn))::numeric, 2) AS revenue_loss_percentage
FROM cleaned_telco_churn
WHERE Churn = 'Yes'
GROUP BY Contract
ORDER BY monthly_revenue_loss DESC;

"""

df_revenue_loss = export_query(query24, 'monthly_contract_revenue_loss')
df_revenue_loss

Exported: monthly_contract_revenue_loss.csv


,contract,monthly_revenue_loss,revenue_loss_percentage
0,Month-to-month,120847.10,26.52
1,One year,14118.45,3.10
2,Two year,4165.30,0.91


Insight:

Typically, Month-to-Month customers cause the highest monthly loss, but losing long-term customers, even if fewer in number, may be more damaging per person.

#### Query 25: Monthly Revenue Loss by Tenure Group

In [65]:
query25 = """
SELECT 
    CASE 
        WHEN tenure BETWEEN 0 AND 6 THEN '0-6 Months'
        WHEN tenure BETWEEN 7 AND 12 THEN '7-12 Months'
        WHEN tenure BETWEEN 13 AND 24 THEN '1-2 Years'
        WHEN tenure BETWEEN 25 AND 48 THEN '2-4 Years'
        ELSE '4+ Years'
    END AS tenure_group,
    SUM(MonthlyCharges) AS monthly_loss,
    ROUND((SUM(MonthlyCharges) * 100.0 / (SELECT SUM(MonthlyCharges) FROM cleaned_telco_churn))::numeric, 2) AS loss_percentage
FROM cleaned_telco_churn
WHERE Churn = 'Yes'
GROUP BY tenure_group
ORDER BY tenure_group;
"""
df_loss_by_tenure = export_query(query25, 'monthly_loss_by_tenure_group')
df_loss_by_tenure

Exported: monthly_loss_by_tenure_group.csv


,tenure_group,monthly_loss,loss_percentage
0,0-6 Months,49896.10,10.95
1,1-2 Years,23081.65,5.07
2,2-4 Years,27462.50,6.03
3,4+ Years,19632.45,4.31
4,7-12 Months,19058.15,4.18


Insight: 

Shows how newer customers churn contributes more to revenue bleeding, strengthening your retention argument.

#### Query 26: Top 5 High-Churn Segments

In [66]:
query26 = """
SELECT 
    Contract,
    PaymentMethod,
    tenure,
    COUNT(*) AS customer_count,
    ROUND(AVG(MonthlyCharges)::numeric, 2) AS avg_monthly_charges,
    ROUND(AVG(TotalCharges)::numeric, 2) AS avg_total_charges
FROM cleaned_telco_churn
WHERE Churn = 'Yes'
GROUP BY Contract, PaymentMethod, tenure
ORDER BY customer_count DESC
LIMIT 5;

"""
df_loss_by_tenure = export_query(query26, 'top5_high_churn_segments')
df_loss_by_tenure

Exported: top5_high_churn_segments.csv


,contract,paymentmethod,tenure,customer_count,avg_monthly_charges,avg_total_charges
0,Month-to-month,Electronic check,1,221,66.92,66.92
1,Month-to-month,Mailed check,1,129,44.46,44.46
2,Month-to-month,Electronic check,2,74,70.66,142.31
3,Month-to-month,Electronic check,3,55,71.12,215.37
4,Month-to-month,Electronic check,4,49,75.35,301.13


Insight: Reveals contract-payment-tenure combinations that lose the most customers.

#### Query 27: Avg Monthly and Total Charges and Avg tenure for churned vs retained customers

In [70]:
query27 = """
SELECT 
    Churn,
    ROUND(AVG(MonthlyCharges)::numeric, 2) AS avg_monthly_charges,
    ROUND(AVG(tenure)::numeric, 2) AS avg_tenure_months,
    ROUND(AVG(TotalCharges)::numeric, 2) AS avg_total_charges
FROM cleaned_telco_churn
GROUP BY Churn
ORDER BY Churn;
"""

df_loss_by_tenure = export_query(query27, 'avg_monthly_total_charges_tenure')
df_loss_by_tenure

Exported: avg_monthly_total_charges_tenure.csv


,churn,avg_monthly_charges,avg_tenure_months,avg_total_charges
0,No,61.31,37.65,2555.34
1,Yes,74.44,17.98,1531.80


#### Query 28: Lifetime Value (LTV) Estimation

In [71]:
query28 = """
SELECT 
    Churn,
    ROUND(AVG(MonthlyCharges * tenure)::numeric, 2) AS avg_lifetime_value
FROM cleaned_telco_churn
GROUP BY Churn
ORDER BY Churn;
"""
df_ltv = export_query(query28, 'avg_lifetime_value_by_churn')
df_ltv


Exported: avg_lifetime_value_by_churn.csv


,churn,avg_lifetime_value
0,No,2555.20
1,Yes,1531.61


Insight: 

Retained customers will show significantly higher LTV, making a strong case for retention investment.

#### Query 29: Churn vs Service Bundle Size

In [72]:
query29 = """
SELECT 
    TotalServices,
    Churn,
    COUNT(*) AS customer_count,
    ROUND(AVG(MonthlyCharges)::numeric, 2) AS avg_monthly_charges,
    ROUND(AVG(TotalCharges)::numeric, 2) AS avg_total_charges,
    ROUND(AVG(tenure)::numeric, 2) AS avg_tenure
FROM cleaned_telco_churn
GROUP BY TotalServices, Churn
ORDER BY TotalServices, Churn;
"""
df_bundle_churn = export_query(query29, 'bundle_vs_churn_charges_tenure')
df_bundle_churn


Exported: bundle_vs_churn_charges_tenure.csv


,totalservices,churn,customer_count,avg_monthly_charges,avg_total_charges,avg_tenure
0,0,No,45,24.96,323.17,12.87
1,0,Yes,35,24.84,91.90,3.77
2,1,No,1338,25.91,574.90,24.99
3,1,Yes,359,45.75,262.32,6.09
4,2,No,796,43.89,1146.63,31.71
5,2,Yes,390,66.02,583.04,9.42
6,3,No,613,65.10,1866.57,29.53
7,3,Yes,352,76.78,1184.42,15.72
8,4,No,632,75.10,2738.42,37.14
9,4,Yes,289,86.75,1784.23,21.04


Insight: 

Customers with fewer services (0–2) churn more, and also have lower tenure + revenue. This supports cross-sell bundling to reduce churn.